In [32]:
import csv
import json
import re, os
import time
import datetime
import copy

dep_id = '6092021'
reward_data_file_path = '../data/{}/reward_data.csv'.format(dep_id)
reward_data_clean_file_path = '../data/{}/reward_data_clean_with_speakerid.csv'.format(dep_id)

1. 6092021:
2. 7132021:
3. 8022021:
4. 1172022: 
5. 2182022: 
6. 3162022: 
7. 9152022: 
8. 9212022:

### Read Reward Data

The saved files of reward_data contains 10 fields:
* TimeSent, TimeReceived', Response, Question, QuestionType, QuestionName, Reactive, SentTimes, ConnectionError, dep_id

In [135]:
## Remove Duplication
with open(reward_data_file_path, newline='') as f:
    with open(reward_data_clean_file_path, 'w') as f_out:
        reward_data_reader = csv.reader(f)
        cnt_line = 0
        prev_line_timesent = None
        prev_line = None
        for line in reward_data_reader:
            cnt_line += 1                           # line 0 represents the head line
            if cnt_line > 1:

                if prev_line_timesent is not None and line[1] == prev_line_timesent:
                    # The current line has the same SentTime as the previous line
                    try:
                        # If this line is same as the previous line, we skip it
                        assert line[0] == prev_line[0]      # speakerID
                        assert line[3] == prev_line[3]      # Response 
                        assert line[4] == prev_line[4]      # Question
                        assert line[5] == prev_line[5]      # Question Type
                        assert line[6] == prev_line[6]      # Question Name
                        assert line[7] == prev_line[7]      # Reactive
                        assert line[8] == prev_line[8]      # SentTimes
                        assert line[9] == prev_line[9]      # ConnectionError
                        assert line[10] == prev_line[10]    # dep-id

                    except:
                        # Otherwise, we still save it into the file
                        print(cnt_line, line)
                        # write this line to the new file
                        f_out.write(','.join([
                            line[0], line[1], line[2], '"'+line[3]+'"', '"'+line[4]+'"',
                            line[5], line[6], line[7], line[8], line[9], line[10]
                            ])
                        )
                        f_out.write('\n')

                    continue

                else:
                    # write this line to the new file
                    f_out.write(','.join([
                        line[0], line[1], line[2], '"'+line[3]+'"', '"'+line[4]+'"',
                        line[5], line[6], line[7], line[8], line[9], line[10]
                        ])
                    )
                    f_out.write('\n')
            else:
                # write the first line (i.e. header) into the file
                f_out.write(','.join(line))
                f_out.write('\n')
            prev_line_timesent = line[1]
            prev_line = copy.deepcopy(line)

60 ['0', '2023-01-19 12:20:33', '0000-00-00 00:00:00.', '-1.0', 'On a scale from 0-10, how stressed or tense do you feel right now? ', 'slide bar', 'baseline:recomm:likertconfirm:1', '0', '1', '0', '1182023']
469 ['9', '2023-01-28 18:10:43', '2023-01-28 20:11:53', '-1', 'If you are experiencing a stressful situation, please click on the button below! ', 'radio', 'request:button:1', '0', '1', '0', '1182023']
502 ['9', '2023-01-29 13:19:53', '2023-01-29 15:04:36', '-1', 'If you are experiencing a stressful situation, please click on the button below! ', 'radio', 'request:button:1', '0', '1', '0', '1182023']
972 ['0', '2023-02-10 20:33:47', '0000-00-00 00:00:00.', '-1.0', 'You have missed a recommendation message at 8:10PM. Check back again later for more recommendation messages!', 'thanks', 'missed:recomm:1', '0', '1', '0', '1182023']
1151 ['9', '2023-02-14 15:09:55', '2023-02-14 18:56:21', '-1', 'If you are experiencing a stressful situation, please click on the button below! ', 'radio'

In [33]:
baseline_period_end_line = 0
with open(reward_data_clean_file_path, newline='') as f_clean:
    print("read file: ", reward_data_clean_file_path)
    reward_data_reader = csv.reader(f_clean)
    cnt_line = 0
    for line in reward_data_reader:
            cnt_line += 1                           # line 0 represents the head line
            if cnt_line > 1:
                question_name = line[6]
                if 'baseline' in question_name:
                    baseline_period_end_line = cnt_line
                if 'daytime' in question_name and 'baseline' not in question_name:
                    if cnt_line >= 100:
                        break

    print('baseline period ends at line: {}'.format(baseline_period_end_line))

read file:  ../../data/6092021/reward_data_clean_with_speakerid.csv
baseline period ends at line: 559


In [ ]:
class RecommMessage():
    def __init__(
        self,
        message_type='',        # Message Type (str), can only be 'proactive', 'reactive', and 'request'
        start_line=-1,
        end_line=-1
    ):
        self.message_type = message_type
        self.start_line = start_line
        self.end_line = end_line

        # check-in (including click the request button)
        self.checkin_send = False           # whether we send the check-in msg
        self.checkin_response = False       # whether we receive the response for the check-in msg
        self.checkin_sendtimes = 0          # initialize the number of times of sending check-in msg to be 0

        # recommendation tips (if request button is clicked, we will send reactive recommendaitons)
        self.recomm_send = False            # whether we send the recommendation tips msg
        self.recomm_response = False        # whether we receive the response for the recommendation tips msg
        self.recomm_sendtimes = 0           # initialize the number of times of sending recommendation tips msg to be 0

        # post-recommendation survey
        self.postrecomm_send = False        # whether we send the post-recommendation survey msg
        self.postrecomm_response = False    # whether we receive the response for the post-recommendation survey msg
        self.postrecomm_sendtimes = 0       # initialize the number of times of sending post-recommendation survey msg to be 0

        # Only for reactive messages
        self.reactive_response_cnt = [0, 0, 0, 0, 0]
        self.reactive_response_answer = [False, False, False, False, False]

    


In [37]:
# Read data files
with open(reward_data_clean_file_path, newline='') as f_clean:
    print("Read file: ", reward_data_clean_file_path)
    print("Baseline period ends at line: {}, we will count proactive/reactive message statistics after that line.".format(baseline_period_end_line))
    reward_data_reader = csv.reader(f_clean)
    cnt_line = 0

    send_recomm_tips_lines = []
    respond_recomm_tips_lines = []

    proactive_recomm = False
    reactive_recomm = False
    request_recomm = False

    proactive_recomm_msgs = []
    reactive_recomm_msgs = []
    request_recomm_msgs = []

    num_proactive_checkins_msg_send = []        # store the number of attemps that we send the check_in:proactive:1
    num_reactive_checkins_msg_send = []         # store the number of attemps that we send the check_in:reactive:1
    num_reactive_2_checkins_msg_send = []       # store the number of attemps that we send the check_in:reactive:2
    num_proactive_checkins_msg_send_with_response = []  # store the number of attemps that we send the check_in:proactive:1 with response
    num_reactive_checkins_msg_send_with_response = []   # store the number of attemps that we send the check_in:reactive:1 with response
    num_reactive_2_checkins_msg_send_with_response = [] # store the number of attemps that we send the check_in:reactive:2 with response

    cnt_proactive_checkins_msg_send = 0         # number of check_in:proactive:1 msgs we send (multiple attemps count as 1)
    cnt_reactive_checkins_msg_send = 0          # number of check_in:reactive:1 msgs we send (multple attemps count as 1)
    cnt_reactive_2_checkins_msg_send = 0        # number of check_in:reactive:2 msgs we send (multiple attemps count as 1)
    cnt_request_button_mgs_send = 0             # number of request button msgs we send
    cnt_proactive_checkins_msg_response = 0     # number of proactive check-in msgs we receive response
    cnt_reactive_checkins_msg_reponse = 0       # number of reactive check-in msgs we receive response
    cnt_request_button_mgs_response = 0         # number of request button msgs we receive response

    cnt_proactive_recomm_send = 0               # number of proactive recomm msg we send (multiple attemps count as 1)
    cnt_proactive_recomm_response = 0           # number of proactive recomm msg we receive response
    cnt_reactive_recomm_send = 0                # number of reactive recomm msg we send (multiple attemps count as 1)
    cnt_reactive_recomm_response = 0            # number of reactive recomm msg we receive response
    cnt_request_recomm_send = 0                 # number of request button recomm msg we send
    cnt_request_recomm_response = 0             # number of request button recomm msg we receive reponse

    num_proactive_recomm_msg_send = []          # store the number of attemps that we send the proactive recomm msgs
    num_reactive_recomm_msg_send = []           # store the number of attemps that we send the reactive recomm msgs
    num_request_recomm_msg_send = []            # store the number of attemps that we send the request button recomm msgs
    num_proactive_recomm_msg_send_w_res = []    # store the number of attemps that we send the proactive recomm msgs with response
    num_reactive_recomm_msg_send_w_res = []     # store the number of attemps that we send the reactive recomm msgs with response
    num_request_recomm_msg_send_w_res = []      # store the number of attemps that we send the request button recomm msgs with response

    cnt_proactive_postrecomm_send = 0           # number of proactive post-recomm msg we send
    cnt_proactive_postrecomm_response = 0       # number of proactive post-recomm msg we receive
    cnt_reactive_postrecomm_send = 0            # number of reactive post-recomm msg we send
    cnt_reactive_postrecomm_response = 0        # number of reactive post-recomm msg we receive
    cnt_request_postrecomm_send = 0             # number of request recomm post-recomm msg we send
    cnt_request_postrecomm_response = 0         # number of request recomm post-recomm msg we receive


    cnt_reactive_response = [0, 0, 0, 0, 0]     # response cnt for the answer -1, 0, 1, 2, 3
    reactive_response_answer = [False, False, False, False, False]
    reactive_check_in_send_times = -1           # how many times we send the reactive chech-in msg before we got an response?
    reactive_check_in_trigger_timestamp = ''    # the triggger event's timestamp, first reactive check-in msg's sent time
    reactive_check_in_trigger_speakerID = 9     # the speaker-id that trigger the reactive check-in msg

    for line in reward_data_reader:
        cnt_line += 1                           # line 0 represents the head line
        if cnt_line != 1:
            speaker_id = line[0]                # speaker ID that trigger this message. Default: 9
            sent_timestamp = line[1]            # timestamp we send the message
            receive_timestamp = line[2]         # timestamp we receive the response (will be all 0 if no response)
            response_str = line[3]              # response of the message
            question_name = line[6]             # question name, str
            sent_times = int(line[8])           # sent times, int

            if cnt_line < baseline_period_end_line:
                # In baseline period, there will be request button msgs, and we will not count them.
                continue

            ''' How can we know that we encounter a new message? There are several conditions.
            1. A new proactive recomm start. We need to first send the proactive check-in msg
                --> question_name is 'daytime:check_in:proactive:1' and send_times = 1
            2. A new reactive recomm start. We need to first send the reactive check=in msg
                --> question_name is 'daytime:check_in:reactive:1' and send_times = 1
            3. All the recommendations in the queue are cleared and we place the request button on the screen.
                --> question_name is 'request:button:1' and send_times = 1 and also we have finished proactive and reactive recomm procedures and we should have proactive_recomm = False and reactive_recomm = False
            4. Missed recommendation msg being send with no respond
                --> question_name is 'missed:recomm:1' and response is not 0
            5. Schduled event msgs being sent, including morning/evening/weekly messages.
                --> question_name contains 'morning', 'evening', or 'weekly'
            '''

            # TODO: Change the current message modeling from list to stack/priority queue

            if ('daytime:check_in:proactive:1' in question_name and sent_times == 1) or \
                ('daytime:check_in:reactive:1' in question_name and sent_times == 1) or \
                    ('request:button' in question_name and sent_times == 1 and proactive_recomm is False and reactive_recomm is False) or \
                        ('morning' in question_name) or ('evening' in question_name) or ('weekly:' in question_name) or \
                            ('missed' in question_name and int(float(response_str)) != 0):

                # If previously we are inside a proactive recommendation procedure
                if proactive_recomm is True:
                    # clear the proactive-recomm msg statistics
                    proactive_recomm_end_line = cnt_line - 1
                    proactive_recomm_msgs.append(
                        [proactive_recomm_start_line, proactive_recomm_end_line,
                         proactive_checkin_response, send_recomm_msg, proactive_recomm_response,
                         send_postrecomm_msg, proactive_postrecomm_response]
                    )
                    if proactive_checkin_response:
                        cnt_proactive_checkins_msg_response += 1
                        num_proactive_checkins_msg_send_with_response.append(num_proactive_checkins_msg_send[-1])
                    if send_recomm_msg:
                        cnt_proactive_recomm_send += 1
                    if proactive_recomm_response:
                        cnt_proactive_recomm_response += 1
                    if send_postrecomm_msg:
                        cnt_proactive_postrecomm_send += 1
                    if proactive_postrecomm_response:
                        cnt_proactive_postrecomm_response += 1

                    if (proactive_checkin_response) and not send_recomm_msg:
                        print('[proactive] check-in msg receive response but no proactive recommendation messages were sent. ', proactive_recomm_start_line, proactive_recomm_end_line, proactive_checkin_response, send_recomm_msg, proactive_recomm_response, send_postrecomm_msg, proactive_postrecomm_response)
                        print('\n')

                    proactive_recomm = False
                    proactive_checkin_response = False
                    send_recomm_msg = False
                    proactive_recomm_response = False
                    send_postrecomm_msg = False
                    proactive_postrecomm_response = False

                # If previously we are inside a reactive recommendation procedure
                if reactive_recomm is True:
                    # cnt the number of different answers to the check_in:reactive:1 msg
                    cnt_ans = 0
                    reactive_check_in_msg_ans = 0
                    for i in range(1,5):
                        if reactive_response_answer[i]:
                            cnt_ans += 1
                    assert cnt_ans < 2      # make sure that at most 1 answer is selected from ans-1 to ans-4
                    if cnt_ans == 1:
                        # we have a response from the check_in:reactive:1 msg
                        for i in range(1,5):
                            # update the counter at the position of the corresponding answer
                            if reactive_response_answer[i]:
                                cnt_reactive_response[i] += 1
                                reactive_check_in_msg_ans = i   # the answer tot he reactive check-in msg
                    else:
                        # we didn't get a response from the check_in:reactive:1 msg 
                        assert reactive_response_answer[0]  # make sure that the position 0 of this vector is True
                        cnt_reactive_response[0] += 1       # update the counter
                        reactive_check_in_msg_ans = 0       # the answer to the reactive check-in msg

                    # clear the reactive-recomm msg statistics
                    reactive_recomm_end_line = cnt_line - 1
                    reactive_recomm_msgs.append(
                        [reactive_recomm_start_line, reactive_recomm_end_line,
                         reactive_checkin_response, send_recomm_msg, reactive_recomm_response,
                         send_postrecomm_msg, reactive_postrecomm_response,
                         reactive_check_in_trigger_speakerID, reactive_check_in_trigger_timestamp,
                         reactive_check_in_send_times, reactive_check_in_msg_ans]
                    )
                    if reactive_checkin_response:
                        # only consider the check_in:reactive:1 msg
                        cnt_reactive_checkins_msg_reponse += 1
                        num_reactive_checkins_msg_send_with_response.append(num_reactive_checkins_msg_send[-1])
                    if send_recomm_msg:
                        cnt_reactive_recomm_send += 1
                    if reactive_recomm_response:
                        cnt_reactive_recomm_response += 1
                    if send_postrecomm_msg:
                        cnt_reactive_postrecomm_send += 1
                    if reactive_postrecomm_response:
                        cnt_reactive_postrecomm_response += 1

                    reactive_recomm = False
                    reactive_checkin_response = False
                    send_recomm_msg = False
                    reactive_recomm_response = False
                    send_postrecomm_msg = False
                    reactive_postrecomm_response = False
                    reactive_response_answer = [False, False, False, False, False]

                # If previously we are inside a request recommendation procedure
                if request_recomm is True:
                    request_recomm_end_line = cnt_line - 1
                    request_recomm_msgs.append(
                        [request_recomm_start_line, request_recomm_end_line,
                         send_recomm_msg, request_recomm_response,
                         send_postrecomm_msg, request_postrecomm_response]
                    )
                    if request_button_response:
                        cnt_request_button_mgs_response += 1
                    if send_recomm_msg:
                        cnt_request_recomm_send += 1
                    if request_recomm_response:
                        cnt_request_recomm_response += 1
                    if send_postrecomm_msg:
                        cnt_request_postrecomm_send += 1
                    if request_postrecomm_response:
                        cnt_request_postrecomm_response += 1

                    if request_button_response:
                        print('[request button] Request button receives response, start line: {}, end line: {}, \
                              send recomm msg: {}, receive recomm response: {}, \
                              send postrecomm msg: {}, receive postrecomm response: {}'.format(
                            request_recomm_start_line, request_recomm_end_line,
                            send_recomm_msg, request_recomm_response,
                            send_postrecomm_msg, request_postrecomm_response
                        ))
                        print('\n')

                    if send_recomm_msg:
                        print('[request button] Request button send recomm msg, start line: {}, end line: {}, \
                              send recomm msg: {}, receive recomm response: {}, \
                              send postrecomm msg: {}, receive postrecomm response: {}'.format(
                            request_recomm_start_line, request_recomm_end_line,
                            send_recomm_msg, request_recomm_response,
                            send_postrecomm_msg, request_postrecomm_response
                        ))
                        print('\n')

                    if send_postrecomm_msg:
                        print('[request button] Request button send post-recomm msg, start line: {}, end line: {}, \
                              send recomm msg: {}, receive recomm response: {}, \
                              send postrecomm msg: {}, receive postrecomm response: {}'.format(
                            request_recomm_start_line, request_recomm_end_line,
                            send_recomm_msg, request_recomm_response,
                            send_postrecomm_msg, request_postrecomm_response
                        ))
                        print('\n')

                    request_recomm = False
                    send_recomm_msg = False
                    request_recomm_response = False
                    send_postrecomm_msg = False
                    request_postrecomm_response = False
                    request_button_response = False

            if 'daytime:check_in:proactive:1' in question_name and sent_times == 1:
                # This is the start of a proactive recomm msg
                proactive_recomm = True
                proactive_recomm_start_line = cnt_line
                # Set the flags to be False
                proactive_checkin_response = False
                send_recomm_msg = False
                proactive_recomm_response = False
                send_postrecomm_msg = False
                proactive_postrecomm_response = False
                cnt_proactive_checkins_msg_send += 1
                num_proactive_checkins_msg_send.append(1)

            if 'daytime:check_in:reactive:1' in question_name and sent_times == 1:
                # This is the start of a reactive recomm msg
                reactive_recomm = True
                reactive_recomm_start_line = cnt_line
                reactive_check_in_send_times = sent_times               # set the sent times of the reactive check-in msg
                reactive_check_in_trigger_timestamp = sent_timestamp    # set the timestamp of sending the check-in msg (1st time)
                reactive_check_in_trigger_speakerID = speaker_id        # set the speaker ID that trigger the check-in msg (1st time)
                # Set the flags to be False
                reactive_checkin_response = False
                send_recomm_msg = False
                reactive_recomm_response = False
                send_postrecomm_msg = False
                reactive_postrecomm_response = False
                reactive_response_answer = [False, False, False, False, False]
                cnt_reactive_checkins_msg_send += 1
                num_reactive_checkins_msg_send.append(1)

            if 'daytime:check_in:reactive:2' in question_name and sent_times == 1:
                # This is the second-level reactive check-in message (i.e. check_in:reactive:2)
                cnt_reactive_2_checkins_msg_send += 1
                num_reactive_2_checkins_msg_send.append(1)

            if 'request:button' in question_name and sent_times == 1:
                # Request Button may be placed during another recommendation procedure, filter out those request buttons
                if proactive_recomm is False and reactive_recomm is False:
                    # This is the start of a request msg (maybe followed by recommendation tips)
                    request_recomm = True
                    request_recomm_start_line = cnt_line
                    # Set the flags to be False
                    request_button_response = False
                    send_recomm_msg = False
                    request_recomm_response = False
                    send_postrecomm_msg = False
                    request_postrecomm_response = False
                    cnt_request_button_mgs_send += 1

            if 'daytime:check_in:proactive:1' in question_name:
                response = int(float(response_str))          # -1 or 0, int
                if response != -1:
                    proactive_checkin_response = True

            if 'daytime:check_in:reactive:1' in question_name:
                response = int(float(response_str))          # -1 to 4, int
                if response == -1:
                    reactive_response_answer[0] = True
                else:
                    reactive_response_answer[response] = True
                    reactive_checkin_response = True
                # When the response of the reactive message is 4, we will send out the reactive recommendation
                if response == 4:
                    print('[Reactive] Receive response with answer 4, will send out recommendaiton tips')
                    print(line)
                    print('\n')

            # Update the number of attempts for different types of check-in messages
            if 'daytime:check_in:proactive:1' in question_name:
                num_proactive_checkins_msg_send[-1] = sent_times
            if 'daytime:check_in:reactive:1' in question_name:
                num_reactive_checkins_msg_send[-1] = sent_times
                reactive_check_in_send_times = sent_times
            if 'daytime:check_in:reactive:2' in question_name:
                num_reactive_2_checkins_msg_send[-1] = sent_times

            if 'request:button' in question_name:
                response = int(float(response_str))
                if response != -1:
                    request_button_response = True

            if 'daytime:recomm:' in question_name:
                # Recommendation messages
                send_recomm_msg = True                  # set the flag to be True
                response = int(float(response_str))          # -1 or 0, int
                if response != -1:
                    # response is 0 means we receive the response from the participant
                    if proactive_recomm:
                        proactive_recomm_response = True
                    if reactive_recomm:
                        reactive_recomm_response = True
                    if request_recomm:
                        request_recomm_response = True

            if 'postrecomm' in question_name:
                # Post-recommendation messages
                send_postrecomm_msg = True              # set the flag to be True
                if 'daytime:postrecomm:implement:1' in question_name:
                    response = int(float(response_str))      # -1 or 0 or 1, int
                    if response != -1:
                        if proactive_recomm:
                            proactive_postrecomm_response = True
                        if reactive_recomm:
                            reactive_postrecomm_response = True
                        if request_recomm:
                            request_postrecomm_response = True
                if 'daytime:postrecomm:helpfulyes:1' in question_name:
                    response = int(float(response_str))
                    if response == -1:
                        print('[post-recomm] Get response of postrecomm:implement:1 msg but no response for the postrecomm:helpfulyes:1 msg')
                        print(line)
                        print('\n')
                if 'daytime:postrecomm:helpfulno:1' in question_name:
                    if response_str[0] != '[':
                        print('[post-recomm] Get response of postrecomm:implement:1 msg but no response for the postrecomm:helpfulno:1 msg')
                        print(line)
                        print('\n')


Read file:  ../../data/6092021/reward_data_clean_with_speakerid.csv
Baseline period ends at line: 559, we will count proactive/reactive message statistics after that line.
[request button] Request button send post-recomm msg, start line: 2157, end line: 2159,                               send recomm msg: False, receive recomm response: False,                               send postrecomm msg: True, receive postrecomm response: False


[request button] Request button receives response, start line: 2515, end line: 2515,                               send recomm msg: False, receive recomm response: False,                               send postrecomm msg: False, receive postrecomm response: False


[request button] Request button receives response, start line: 4536, end line: 4536,                               send recomm msg: False, receive recomm response: False,                               send postrecomm msg: False, receive postrecomm response: False




In [35]:
import numpy as np

assert len(num_proactive_checkins_msg_send) == cnt_proactive_checkins_msg_send
assert len(num_proactive_checkins_msg_send_with_response) == cnt_proactive_checkins_msg_response
assert len(num_reactive_checkins_msg_send) == cnt_reactive_checkins_msg_send
assert len(num_reactive_checkins_msg_send_with_response) == cnt_reactive_checkins_msg_reponse

print('===='*4 + ' Check-in Msgs ' + '===='*4)
print('[Proactive check-in] Average attemps: {}, Average attemps (with response): {}'.format(
    np.mean(num_proactive_checkins_msg_send), np.mean(num_proactive_checkins_msg_send_with_response)
))
print('[Reactive check-in] Average attemps: {}, Average attemps (with response): {}'.format(
    np.mean(num_reactive_checkins_msg_send), np.mean(num_reactive_checkins_msg_send_with_response)
))

================ Check-in Msgs ================
[Proactive check-in] Average attemps: 2.3276595744680852, Average attemps (with response): 1.7906976744186047
[Reactive check-in] Average attemps: 2.1818181818181817, Average attemps (with response): 1.7428571428571429


In [36]:
print('===='*4 + ' Check-in Msgs ' + '===='*4)
print('[Proactive check-in] send: {}, response: {}'.format(cnt_proactive_checkins_msg_send, cnt_proactive_checkins_msg_response))
print('[Reactive check-in] send: {}, response: {}'.format(cnt_reactive_checkins_msg_send, cnt_reactive_checkins_msg_reponse))
print('[Request button] send: {}, response: {}'.format(cnt_request_button_mgs_send, cnt_request_button_mgs_response))
print('\n')

print('===='*4 + ' Recomm Msgs ' + '===='*4)
print('[Proactive recomm] send: {}, response: {}'.format(cnt_proactive_recomm_send, cnt_proactive_recomm_response))
print("[Reactive recomm] send: {}, response: {}".format(cnt_reactive_recomm_send, cnt_reactive_recomm_response))
print("[Request recomm] send: {}, response: {}".format(cnt_request_recomm_send, cnt_request_recomm_response))
print('\n')

print('===='*4 + ' Post-recomm Msgs ' + '===='*4)
print('[Proactive postrecomm] send: {}, response: {}'.format(cnt_proactive_postrecomm_send, cnt_proactive_postrecomm_response))
print("[Reactive postrecomm] send: {}, response: {}".format(cnt_reactive_postrecomm_send, cnt_reactive_postrecomm_response))
print("[Request postrecomm] send: {}, response: {}".format(cnt_request_postrecomm_send, cnt_request_postrecomm_response))
print('\n')

================ Check-in Msgs ================
[Proactive check-in] send: 235, response: 86
[Reactive check-in] send: 77, response: 35
[Request button] send: 560, response: 2


================ Recomm Msgs ================
[Proactive recomm] send: 86, response: 86
[Reactive recomm] send: 15, response: 15
[Request recomm] send: 0, response: 0


================ Post-recomm Msgs ================
[Proactive postrecomm] send: 78, response: 20
[Reactive postrecomm] send: 15, response: 6
[Request postrecomm] send: 1, response: 0




In [140]:
len(reactive_recomm_msgs)

294

In [141]:
''' Fields of reactive_recomm_msgs
0: reactive_recomm_start_line               # start line of the reactive recomm msg
1: reactive_recomm_end_line                 # end line of the reactive recomm msg 
2: reactive_checkin_response                # whether the reactive check-in msg receive response
3: send_recomm_msg                          # whether the recomm msg is send
4: reactive_recomm_response                 # whether the reactive recomm msg receive response
5: send_postrecomm_msg                      # whether the post-recomm msg is send
6: reactive_postrecomm_response             # whether the reactive post-recomm msg receive response
7: reactive_check_in_trigger_speakerID      # speaker ID that trigger the reactive check-in msg
8: reactive_check_in_trigger_timestamp      # timestamp that we send the first reactive check-in msg
9: reactive_check_in_send_times             # how many times the reactive check-in msg was sent before respond
10: reactive_check_in_msg_ans               # answer to the reactive check-in msg. 0: no response. 1-4: answer
'''

cnt_reactive_check_in_send_by_senttimes = {1: 0, 2: 0, 3: 0}
cnt_reactive_check_in_response_by_senttimes = {1: 0, 2: 0, 3: 0}
cnt_reactive_check_in_answer_by_senttimes = {
    1: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
    2: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
    3: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
}
reactive_speakerID_0_timestamp_with_ans = []


for reactive_msg_stat in reactive_recomm_msgs:
    reactive_msg_speakerID = reactive_msg_stat[7]
    if reactive_msg_speakerID != '0':
        print(reactive_msg_stat)
        continue
    reactive_msg_sendtime = reactive_msg_stat[-2]
    reactive_msg_ans = reactive_msg_stat[-1]
    reactive_msg_timestamp = reactive_msg_stat[-3]
    if reactive_msg_stat[2]:    # reactive_checkin_response
        cnt_reactive_check_in_response_by_senttimes[reactive_msg_sendtime] += 1
    cnt_reactive_check_in_send_by_senttimes[reactive_msg_sendtime] += 1
    cnt_reactive_check_in_answer_by_senttimes[reactive_msg_sendtime][reactive_msg_ans] += 1

    reactive_speakerID_0_timestamp_with_ans.append(
        [reactive_msg_timestamp, reactive_msg_ans, reactive_msg_sendtime]
    )

[987, 988, True, False, False, False, False, '9', '2023-02-11 09:24:38', 1, 3]
[989, 990, True, False, False, False, False, '1', '2023-02-11 09:58:04', 2, 3]
[994, 996, False, False, False, False, False, '1', '2023-02-11 11:04:41', 3, 0]
[1003, 1004, False, False, False, False, False, '1', '2023-02-11 15:10:56', 2, 0]
[1005, 1007, False, False, False, False, False, '1', '2023-02-11 15:24:41', 2, 0]
[1008, 1011, False, False, False, False, False, '1', '2023-02-11 16:05:21', 2, 0]
[1019, 1019, False, False, False, False, False, '1', '2023-02-11 21:25:07', 1, 0]
[1046, 1047, False, False, False, False, False, '1', '2023-02-12 17:21:24', 2, 0]
[1048, 1050, False, False, False, False, False, '1', '2023-02-12 17:40:16', 2, 0]
[1051, 1053, False, False, False, False, False, '1', '2023-02-12 18:08:23', 2, 0]
[1058, 1060, False, False, False, False, False, '1', '2023-02-12 20:23:29', 2, 0]
[1075, 1077, True, False, False, False, False, '9', '2023-02-13 10:27:59', 2, 3]
[1087, 1089, False, False

In [142]:
reactive_speakerID_0_timestamp_file_path = '../../data/{}/reactive_timestamp_with_ans_sendtime.csv'.format(dep_id)
with open(reactive_speakerID_0_timestamp_file_path, 'w') as f_out:
    f_out.write('timestamp' + ',' + 'response' + ',' + 'sendtime' + '\n')
    for timestamp in reactive_speakerID_0_timestamp_with_ans:
        f_out.write(timestamp[0]+','+str(timestamp[1])+','+str(timestamp[2]))
        f_out.write('\n')

In [143]:
cnt_reactive_check_in_send_by_senttimes

{1: 5, 2: 120, 3: 20}

In [144]:
cnt_reactive_check_in_response_by_senttimes

{1: 4, 2: 2, 3: 3}

In [145]:
reactive_check_in_response_rate_by_senttimes = {1: 0.0, 2: 0.0, 3: 0.0}
for i in range(1, 4):
    if cnt_reactive_check_in_send_by_senttimes[i] == 0:
        continue
    reactive_check_in_response_rate_by_senttimes[i] = (
        cnt_reactive_check_in_response_by_senttimes[i] / cnt_reactive_check_in_send_by_senttimes[i]
    )

In [146]:
print("[Reactive check-in msg send by sent times] 1: {}, 2: {}, 3: {}".format(
    cnt_reactive_check_in_send_by_senttimes[1],
    cnt_reactive_check_in_send_by_senttimes[2],
    cnt_reactive_check_in_send_by_senttimes[3],
))
print("[Reactive check-in msg response by sent times] 1: {}, 2: {}, 3: {}".format(
    cnt_reactive_check_in_response_by_senttimes[1],
    cnt_reactive_check_in_response_by_senttimes[2],
    cnt_reactive_check_in_response_by_senttimes[3]
))
print("[Reactive check-in msg response rate by sent times] 1: {}, 2: {}, 3: {}".format(
    reactive_check_in_response_rate_by_senttimes[1],
    reactive_check_in_response_rate_by_senttimes[2],
    reactive_check_in_response_rate_by_senttimes[3],
))

[Reactive check-in msg send by sent times] 1: 5, 2: 120, 3: 20
[Reactive check-in msg response by sent times] 1: 4, 2: 2, 3: 3
[Reactive check-in msg response rate by sent times] 1: 0.8, 2: 0.016666666666666666, 3: 0.15


In [147]:
cnt_reactive_check_in_answer_by_senttimes

{1: {0: 1, 1: 0, 2: 0, 3: 3, 4: 1},
 2: {0: 118, 1: 0, 2: 0, 3: 2, 4: 0},
 3: {0: 17, 1: 0, 2: 0, 3: 2, 4: 1}}

## Reactive Message
### Only Select Reactive Messages with speakerID=0

In [83]:
reward_data_file_path = '../../data/{}/reward_data_clean_with_speakerid.csv'.format(dep_id)
print("Read file: ", reward_data_file_path)
with open(reward_data_file_path, newline='') as f:
    reward_data_reader = csv.reader(f)
    cnt_line = 0
    reactive_speakerID_0_timestamp = []

    for line in reward_data_reader:
        cnt_line += 1
        if cnt_line != 1:
            speakerID = int(line[0])
            timesent_timestamp = line[1]
            question_name = line[6]
            reactive_flag = int(line[7])
            sent_times = int(line[8])
            if 'reactive:1' in question_name and speakerID==0 and sent_times==1:
                response = int(float(line[3]))
                if response!=-1:
                    reactive_speakerID_0_timestamp.append([timesent_timestamp, response])

print(reactive_speakerID_0_timestamp)
print(len(reactive_speakerID_0_timestamp))

[['2022-10-06 13:20:55', 1], ['2022-10-07 13:10:58', 3], ['2022-10-07 17:40:29', 3], ['2022-10-09 16:03:59', 2], ['2022-10-09 16:07:56', 3], ['2022-10-12 17:26:53', 1], ['2022-10-13 13:06:22', 3], ['2022-10-13 17:34:30', 1], ['2022-10-17 12:34:39', 3], ['2022-10-17 19:21:57', 3], ['2022-10-17 19:25:11', 3], ['2022-10-18 12:08:34', 3], ['2022-10-21 13:27:43', 3], ['2022-10-24 12:16:34', 2], ['2022-10-25 12:29:30', 3], ['2022-10-26 13:25:10', 3], ['2022-10-26 19:09:46', 3], ['2022-10-27 17:38:08', 1], ['2022-11-02 13:36:36', 3], ['2022-11-02 13:45:27', 3], ['2022-11-09 14:02:30', 3], ['2022-11-10 14:19:46', 3], ['2022-11-18 10:05:37', 1], ['2022-11-18 12:09:36', 2], ['2022-11-18 12:13:20', 3], ['2022-11-30 11:14:36', 3], ['2022-11-30 11:16:23', 3], ['2022-11-30 11:23:48', 3], ['2022-11-30 11:39:49', 3], ['2022-11-30 12:04:50', 3], ['2022-12-08 13:28:07', 2], ['2022-12-14 10:38:59', 3], ['2022-12-14 10:42:28', 3], ['2022-12-14 11:29:17', 3], ['2022-12-19 12:20:48', 3], ['2022-12-23 14:48:

In [84]:
reactive_speakerID_0_timestamp_file_path = '../../data_speakerID/{}/reactive_timestamp.csv'.format(dep_id)
with open(reactive_speakerID_0_timestamp_file_path, 'w') as f_out:
    for timestamp in reactive_speakerID_0_timestamp:
        # print(timestamp)
        # print(str(timestamp[1]))
        f_out.write(timestamp[0]+','+str(timestamp[1]))
        f_out.write('\n')